In [1]:
import sys
sys.path.append('../')   # If you move this notebook to another location, change this path to point to the root directory of the solutions project

from pathlib import Path
import warnings
from integrations import elc_integration

warnings.simplefilter("ignore")

In [3]:
import importlib
importlib.reload(elc_integration)

<module 'integrations.elc_integration' from 'C:\\Working\\solutions\\integrations\\..\\integrations\\elc_integration.py'>

In [4]:
elc_integration.gather_emissions_factors_vmas()

{'coal': <model.vma.VMA at 0x1eb7335f580>,
 'natural gas': <model.vma.VMA at 0x1eb7335f130>,
 'hydro': <model.vma.VMA at 0x1eb7088a0d0>,
 'oil products': <model.vma.VMA at 0x1eb7088f040>,
 'onshorewind': <model.vma.VMA at 0x1eb7337b7f0>,
 'offshorewind': <model.vma.VMA at 0x1eb7337bfd0>,
 'solarpvutil': <model.vma.VMA at 0x1eb73386880>,
 'solarpvroof': <model.vma.VMA at 0x1eb73386d60>,
 'concentratedsolar': <model.vma.VMA at 0x1eb733937c0>,
 'geothermal': <model.vma.VMA at 0x1eb73393850>,
 'waveandtidal': <model.vma.VMA at 0x1eb733a2760>,
 'biomass': <model.vma.VMA at 0x1eb733a2b80>,
 'microwind': <model.vma.VMA at 0x1eb733ad6a0>,
 'instreamhydro': <model.vma.VMA at 0x1eb733adcd0>,
 'nuclear': <model.vma.VMA at 0x1eb733ad430>,
 'wastetoenergy': <model.vma.VMA at 0x1eb733bab50>,
 'landfillmethane': <model.vma.VMA at 0x1eb733ba670>,
 'biogas': <model.vma.VMA at 0x1eb733c89a0>}

# One-time Code Used to Construct Original 

In [2]:
from tools.vma_xls_extract import VMAReader
from tools import util
from model import sma
from tools import sma_xls_extract
import openpyxl

In [3]:
excel_file = Path(r'C:\Working\Integrations\2020_Project Drawdown_ELC Sector_Integration Model (August2020)_v11.xlsx')
wb = openpyxl.load_workbook(excel_file, data_only=True, keep_links=False)

In [3]:
# save the first five VMAs on tab 2A, for the conventional technologies.
# Note: I had to manually add the "Exclude Data?"  header to column Q to get this to work.

vmar = VMAReader(wb)
dfdict = vmar.xls_df_dict(sheetname='2A - Emissions Factors_VMA', fixed_summary=True, start_row=13, max_tables=5)

In [4]:
for title in dfdict.keys():
    path_friendly_title = util.to_filename(title) + '.csv'
    fullpath = Path("data/elc") / path_friendly_title
    (df,use_weights,stats) = dfdict[title]
    df.to_csv(fullpath)
    print(f"| {title}| {fullpath} | {use_weights} | {stats} ")

| OIL Emissions Factor| data\elc\OIL_Emissions_Factor.csv | False | (828.22, 1047.5515152913504, 608.8884847086497) 
| COAL_Emissions Factor| data\elc\COAL_Emissions_Factor.csv | False | (1028.25, 1250.948609519274, 805.551390480726) 
| NATURAL GAS_Emissions Factor| data\elc\NATURAL_GAS_Emissions_Factor.csv | False | (574.0743333333334, 746.9153177801815, 401.23334888648526) 
| NUCLEAR_Indirect CO2 Emissions| data\elc\NUCLEAR_Indirect_CO2_Emissions.csv | False | (18.459400000000002, 45.196413177241766, 0.8) 
| HYDRO_Indirect CO2 Emissions| data\elc\HYDRO_Indirect_CO2_Emissions.csv | False | (202.26478260869567, 703.9449847283984, 0.0) 


In [5]:
wb.close()

In [4]:
# Read the conventional adoption SMA from tabl 5A, columns AG-AW

adoption = sma_xls_extract.extract_sma(wb, sheet_name='5A- ADOPTION_Conventional+Hydro', year_col='AF')

In [9]:
adoption.region_cases

{'Not found 15': {'Baseline Cases': ['S1', 'S2', 'S3', 'S4', 'S5'],
  'Conservative Cases': ['S6', 'S7', 'S8'],
  'Ambitious Cases': ['S9', 'S10', 'S11', 'S12', 'S13'],
  '100% RES2050 Cases': ['S14', 'S15']},
 'Not found 74': {'Baseline Cases': ['S1', 'S2', 'S3', 'S4', 'S5'],
  'Conservative Cases': ['S6', 'S7', 'S8'],
  'Ambitious Cases': ['S9', 'S10', 'S11', 'S12', 'S13'],
  '100% RES2050 Cases': ['S14', 'S15']},
 'Not found 130': {'Baseline Cases\t\t\t': ['S1', 'S2', 'S3', 'S4', 'S5'],
  'Conservative Cases': ['S6', 'S7', 'S8'],
  'Ambitious Cases': ['S9', 'S10', 'S11', 'S12', 'S13', 'S16'],
  '100% RES2050 Cases': ['S14', 'S15', 'S16']},
 'Not found 187': {'Baseline Cases\t\t\t': ['S1', 'S2', 'S3', 'S4', 'S5'],
  'Conservative Cases': ['S6', 'S7', 'S8'],
  'Ambitious Cases': ['S9', 'S10', 'S11', 'S12', 'S13'],
  '100% RES2050 Cases': ['S14']}}

In [5]:
adoption.rename_region('Not found 15','coal')
adoption.rename_region('Not found 74','natural gas')
adoption.rename_region('Not found 130','hydro')
adoption.rename_region('Not found 187','oil products')

In [6]:
adoption.sources['S1'].data.head()

,coal,natural gas,hydro,oil products
Year,,,,
2012,10416.857143,6458.0,3683.0,1260.0
2013,10692.095238,6675.0,3787.0,1240.0
2014,10953.142857,6892.0,3904.0,1220.0
2015,11200.000000,7097.0,3897.0,1200.0
2016,11432.666667,7302.0,4049.0,1180.0


In [7]:
adoption.write(r'data\elc','conventional_adoption')